In [48]:
# DO NOT modify this cell. 
filename = "Buchtel.pgm"
verticalSeam2Remove = 100
horizontalSeam2Remove = 0

# Yes, you can create your own test cases and you should. Do it in a new cell.

In [49]:
# filename = "CAS_2.pgm"
# verticalSeam2Remove = 100
# horizontalSeam2Remove = 50

In [50]:
import queue

def read():
    file = open(filename, "r")
    pgmNumbers = queue.Queue()

    P2 = file.readline()[:-1]

    comments = []
    temp = "#"
    while temp[0] == "#":
        temp = file.readline()[:-1]
        if temp[0] != "#":
            dimensions = temp.split()
        else:
            comments.append(temp)

    columns = int(dimensions[0])
    rows = int(dimensions[1])

    grayscale = int(file.readline())

    for line in file:
        for number in line.split():
            pgmNumbers.put(int(number))

    matrix = [[0 for _ in range(columns)] for _ in range(rows)]

    # Fill the matrix with numbers from the queue
    for i in range(rows):
        for j in range(columns):
            matrix[i][j] = pgmNumbers.get()

    file.close()

    return [P2, comments, columns, rows, grayscale, matrix]

def write(filename, P2, comments, columns, rows, grayscale, matrix):
    file = open(filename, "w")

    file.write(P2 + "\n")
    for comment in comments:
        file.write(comment + "\n")
    file.write(str(columns) + " " + str(rows) + "\n")
    file.write(str(grayscale) + "\n")

    for i in range(rows):
        for j in range(columns):
            file.write(str(matrix[i][j]) + " ")
        file.write("\n")

    file.close()

output = read()
P2 = output[0]
comments = output[1]
numColumns = output[2]
numRows = output[3]
grayscale = output[4]
matrix = output[5]

# write("output.pgm", P2, comments, numColumns, numRows, grayscale, matrix)


In [51]:
# Part I : Vertical seam removal only
# Save your processed file to img_processed_v_h.pgm
filename1 = filename.split(".")[0]+"_processed_"+str(verticalSeam2Remove)+"_0.pgm"


In [52]:
import numpy

#note matrix is defined above

def getEnergy():

    #add a border to the image
    image = numpy.pad(matrix, pad_width=1, mode='edge')


    # Shift the image in different directions
    up = numpy.roll(image, -1, axis=0)
    down = numpy.roll(image, 1, axis=0)
    left = numpy.roll(image, -1, axis=1)
    right = numpy.roll(image, 1, axis=1)

    # Calculate the energy
    energy = numpy.abs(image - up) + numpy.abs(image - down) + numpy.abs(image - right) + numpy.abs(image - left)

    # Remove the border
    energy = energy[1:-1, 1:-1]

    return energy


In [53]:
for _ in range(verticalSeam2Remove):
    energy = getEnergy()

    # calculate cumulative energy matrix
    cumulative = energy

    # leave the first row as it is
    for i in range(1, len(energy)):
        for j in range(len(energy[i])):
            if(j-1<0):
                left = float("inf")
            else:
                left = cumulative[i-1][j-1]

            if(j+1>=len(energy[i])):
                right = float("inf")
            else:
                right = cumulative[i-1][j+1]

            up = cumulative[i-1][j]

            cumulative[i][j] = energy[i][j] + min(left, right, up)

    # Initialize the seam with the index of the minimum value in the last row
    seam = [numpy.argmin(cumulative[-1])]

    # Trace the seam from the bottom to the top
    for i in range(len(cumulative) - 2, -1, -1):
        j = seam[-1]

        # If we are not at the left edge
        if j > 0:
            # If we are not at the right edge
            if j < len(cumulative[0]) - 1:
                # Find the index of the minimum value in the 3-pixel neighborhood
                idx = numpy.argmin(cumulative[i, j-1:j+2])
                seam.append(j + idx - 1)
            else:
                # Find the index of the minimum value in the 2-pixel neighborhood
                idx = numpy.argmin(cumulative[i, j-1:j+1])
                seam.append(j + idx - 1)
        else:
            # Find the index of the minimum value in the 2-pixel neighborhood
            idx = numpy.argmin(cumulative[i, j:j+2])
            seam.append(j + idx)

    # Reverse the seam because we traced it from the bottom to the top
    seam = seam[::-1]

    # Remove the seam from the image
    # Create an array of row indices
    matrix = numpy.array(matrix)
    rows = numpy.arange(matrix.shape[0])

    # Create a mask that is True for all pixels that are not in the seam
    mask = numpy.ones(matrix.shape, dtype=bool)
    mask[rows, seam] = False

    # Use the mask to create a new matrix without the seam
    matrix = matrix[mask].reshape(matrix.shape[0], -1)

In [54]:
write(filename1, P2, comments, numColumns-verticalSeam2Remove, numRows, grayscale, matrix)


In [55]:
# Part II : both vertical and horizontal seams removal 
# Save your processed file to img_processed_v_h.pgm
filename2 = filename.split(".")[0]+"_processed_"+ \
            str(verticalSeam2Remove)+"_"+str(horizontalSeam2Remove)+".pgm"
print(filename2)


Buchtel_processed_100_0.pgm


In [56]:
# Transpose the matrix
matrix = matrix.T


In [57]:
for _ in range(horizontalSeam2Remove):
    energy = getEnergy()

    # calculate cumulative energy matrix
    cumulative = energy

    # leave the first row as it is
    for i in range(1, len(energy)):
        for j in range(len(energy[i])):
            if(j-1<0):
                left = float("inf")
            else:
                left = cumulative[i-1][j-1]

            if(j+1>=len(energy[i])):
                right = float("inf")
            else:
                right = cumulative[i-1][j+1]

            up = cumulative[i-1][j]

            cumulative[i][j] = energy[i][j] + min(left, right, up)

    # Initialize the seam with the index of the minimum value in the last row
    seam = [numpy.argmin(cumulative[-1])]

    # Trace the seam from the bottom to the top
    for i in range(len(cumulative) - 2, -1, -1):
        j = seam[-1]

        # If we are not at the left edge
        if j > 0:
            # If we are not at the right edge
            if j < len(cumulative[0]) - 1:
                # Find the index of the minimum value in the 3-pixel neighborhood
                idx = numpy.argmin(cumulative[i, j-1:j+2])
                seam.append(j + idx - 1)
            else:
                # Find the index of the minimum value in the 2-pixel neighborhood
                idx = numpy.argmin(cumulative[i, j-1:j+1])
                seam.append(j + idx - 1)
        else:
            # Find the index of the minimum value in the 2-pixel neighborhood
            idx = numpy.argmin(cumulative[i, j:j+2])
            seam.append(j + idx)

    # Reverse the seam because we traced it from the bottom to the top
    seam = seam[::-1]

    # Remove the seam from the image
    # Create an array of row indices
    matrix = numpy.array(matrix)
    rows = numpy.arange(matrix.shape[0])

    # Create a mask that is True for all pixels that are not in the seam
    mask = numpy.ones(matrix.shape, dtype=bool)
    mask[rows, seam] = False

    # Use the mask to create a new matrix without the seam
    matrix = matrix[mask].reshape(matrix.shape[0], -1)

In [58]:
matrix = matrix.T

In [59]:
write(filename2, P2, comments, numColumns-verticalSeam2Remove, numRows-horizontalSeam2Remove, grayscale, matrix)
